In [13]:
import numpy as np

## 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        #### 시작 ####
        self.x = x
        self.y = y
        out = x * y
        #### 끝 ####
        return out
    
    def backward(self, dout):
        #### 시작 ####
        dx = dout * self.y
        dy = dout * self.x
        #### 끝 ####
        return dx, dy

In [8]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
assert int(price) == 220, '오답입니다.'
assert mul_apple_layer.x == 100, '캐시가 저장되지 않았습니다'

In [11]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

assert dapple == 2.2, '오답입니다.'
assert int(dapple_num) == 110, '오답입니다.'
assert dtax == 200, '오답입니다.'

## Relu 계층

$$y=
\begin{cases}
x & (x > 0)\\
0 & (x \leq 0)
\end{cases}

In [12]:
class Relu:
    def __init__(self):
        '''
        특정 값들만을 활성화 또는 비활성화하는 테크닉으로 mask가 자주 사용됩니다.
        이를 활용하여 Relu의 forward와 bacwkard 메서드를 구현하세요.
        '''
        self.mask = None
    
    def forward(self, x):
        #### 시작 ####
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        #### 끝 ####
        return out
    
    def backward(self, dout):
        #### 시작 ####
        dout[self.mask] = 0
        dx = dout
        #### 끝 ####
        return dx

In [24]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])

relu = Relu()
assert np.allclose(relu.forward(x), np.array([[1., 0.], [0., 3.]])), '오답입니다.'
assert np.allclose(relu.mask, np.array([[False,  True], [ True, False]])), 'mask를 제대로 활용하지 못했습니다.'

dout = np.ones((2,2))
assert np.allclose(relu.backward(dout), np.array([[1., 0.],[0., 1.]])), 'backward가 제대로 구현되지 않았습니다.'

## Sigmoid 계층

$$y=\frac{1}{1+exp(-x)}$$

In [25]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        #### 시작 ####
        out = 1 / (1 + np.exp(-x))
        self.out = out
        #### 끝 ####
        return out
    
    def backward(self, dout):
        #### 시작 ####
        dx = dout * (1.0 - self.out) * self.out
        #### 끝 ####
        return dx

In [32]:
sigmoid = Sigmoid()
x = np.array([[1.0, -0.5], [-2.0, 3.0]])

out = sigmoid.forward(x)
assert np.allclose(out, np.array([[0.73105858, 0.37754067], [0.11920292, 0.95257413]])), '오답입니다.'

dout = np.ones((2,2))
dx = sigmoid.backward(dout)
assert np.allclose(dx, np.array([[0.19661193, 0.23500371], [0.10499359, 0.04517666]]))

## Affine 계층

In [33]:
class Affine:
    '''
    어떤 변수를 초기화 해둬야 할지에 대해서도 잘 생각해 보세요.
    그리고 bacwkard를 수행할 때 주의할 점은 batch 단위를 고려해야 한다는 점입니다.
    여러 개의 입력이 한 번에 주어질 땐 어떤 연산을 해야 할지를 잘 생각해 봐야 합니다.
    '''
    def __init__(self, W, b):
        self.W = W
        self.b = b
        #### 시작 ####
        self.x = None
        self.dW = None
        self.db = None
        #### 끝 ####

    def forward(self, x):
        #### 시작 ####
        self.x = x
        out = np.dot(x, self.W) + self.b
        #### 끝
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [63]:
np.random.seed(42)

X = np.array([[0,1,2], [7,8,9]])
W = np.random.randn(3,4)
B = np.array([1,2,3,4])

affine = Affine(W, B)
y = affine.forward(X)

assert np.allclose(y ,np.array([[-0.17310215,  2.85098313,  3.65237743,  3.83597522], 
                      [-1.6214974 ,  4.04209463, 15.99676306, 16.60911905]])), '오답입니다.'
assert np.allclose(affine.x, np.array(([[0, 1, 2], [7, 8, 9]]))), '오답입니다.'

dx = affine.backward(np.random.randn(2,4))
assert np.allclose(dx, np.array([[-1.58886576, -2.76421799, -0.09043303],
                            [-3.28563423, -2.35423325,  1.72454259]])), '오답입니다.'
assert np.allclose(affine.dW, np.array([[ -7.08981784,   2.19973133,  -6.35616853,  -9.88612591],
       [ -7.86068669,   0.60069842,  -8.98911044, -11.86071714],
       [ -8.63155554,  -0.9983345 , -11.62205234, -13.83530837]])), '오답입니다.'